In [1]:
!conda activate DS807

In [46]:
import data_loader as dl

In [47]:
df = dl.load_1d_grays()
df

,index,string,CC,D,Y,gray_value,filename
0,1,1836,0,3,6,"[[[166.0], [167.0], [166.0], [162.0], [164.0],...",1.jpg
1,2,1836,0,3,6,"[[[169.0], [168.0], [169.0], [170.0], [166.0],...",2.jpg
2,3,1840,0,4,0,"[[[167.0], [169.0], [169.0], [166.0], [165.0],...",3.jpg
3,4,1840,0,4,0,"[[[167.0], [165.0], [165.0], [165.0], [163.0],...",4.jpg
4,5,1823,0,2,3,"[[[62.0], [45.0], [45.0], [37.0], [40.0], [33....",5.jpg
...,...,...,...,...,...,...,...
11995,11996,1808,0,0,8,"[[[88.0], [62.0], [60.0], [68.0], [61.0], [101...",11996.jpg
11996,11997,1809,0,0,9,"[[[173.0], [171.0], [173.0], [175.0], [174.0],...",11997.jpg
11997,11998,1840,0,4,0,"[[[189.0], [189.0], [189.0], [189.0], [189.0],...",11998.jpg
11998,11999,1840,0,4,0,"[[[191.0], [192.0], [192.0], [192.0], [193.0],...",11999.jpg


In [48]:
df['gray_value'][0].shape

(56, 106, 1)

In [49]:

import numpy as np
arr = np.zeros(shape=(12000,56,106))
i = 0
while i < 12000:
    arr[i] = np.squeeze(df['gray_value'][i])
    i += 1
arr.shape


(12000, 56, 106)

In [50]:
n_samples = len(arr)
x_values = arr.reshape((n_samples, -1))
x_values.shape

(12000, 5936)

In [51]:
len(df['CC'])

12000

In [52]:
from sklearn.model_selection import train_test_split

y_values = df['CC']
x_train, x_test, y_train, y_test = train_test_split(x_values, y_values, test_size=0.2, random_state=2022)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2022)
print(x_train.shape, x_val.shape, x_test.shape)


(7680, 5936) (1920, 5936) (2400, 5936)


In [54]:
x_train = x_train/255
x_val = x_val/255
x_test = x_test/255

In [8]:
from sklearn import svm
from sklearn.metrics import accuracy_score

kernels = ['linear', 'poly', 'rbf', 'sigmoid']
Cs = [1,2,3]
Ds = [1,2,3,4]
gammas = ['scale','auto']
decisions = ['ovr', 'ovo']
results = []

for kernel in kernels:
        if kernel == 'poly':
            for C in Cs:
                for D in Ds:
                    for gamma in gammas:
                        for decision in decisions:
                            svm_current = svm.SVC(kernel=kernel, 
                                                    degree=D,
                                                    C =C,
                                                    gamma = gamma,
                                                    random_state = 2022,
                                                    decision_function_shape = decision)
                            svm_current.fit(x_train, y_train)
                            y_val_hat = svm_current.predict(x_val)
                            accuracy = accuracy_score(y_val_hat, y_val)
                            #
                            results.append([accuracy, kernel, C, gamma, decision, D])
        else:
            for C in Cs:
                for gamma in gammas:
                    for decision in decisions:
                        svm_current = svm.SVC(kernel=kernel,
                                              C=C,
                                              gamma = gamma,
                                              decision_function_shape = decision,
                                              random_state = 2022)
                        svm_current.fit(x_train, y_train)
                        y_val_hat = svm_current.predict(x_val)
                        accuracy = accuracy_score(y_val_hat, y_val)
                        #
                        results.append([accuracy, kernel, C, gamma, decision, None])

results = pd.DataFrame(results)
results.columns = ['Accuracy', 'Kernel','C', 'Gamma', 'Decision', 'Degree']


KeyboardInterrupt: 

In [32]:
import pandas as pd
results = pd.DataFrame(results)
results.columns = ['Accuracy', 'Kernel', 'C', 'Gamma', 'Decision', 'Degree']

results

,Accuracy,Kernel,C,Gamma,Decision,Degree
0,0.973437,linear,1,scale,ovr,NaN
1,0.973437,linear,1,scale,ovo,NaN
2,0.973437,linear,1,auto,ovr,NaN
3,0.973437,linear,1,auto,ovo,NaN
4,0.973437,linear,2,scale,ovr,NaN
...,...,...,...,...,...,...
79,0.966667,sigmoid,2,auto,ovo,NaN
80,0.966667,sigmoid,3,scale,ovr,NaN
81,0.966667,sigmoid,3,scale,ovo,NaN
82,0.966667,sigmoid,3,auto,ovr,NaN


In [33]:
#Print results
print(str(results[results['Accuracy'] == results['Accuracy'].max()]))

    Accuracy Kernel  C  Gamma Decision  Degree
68  0.982292    rbf  3  scale      ovr     NaN
69  0.982292    rbf  3  scale      ovo     NaN


In [34]:
results.to_csv('SVM_results')

In [1]:
import pandas as pd
from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np

In [56]:
svm_final = svm.SVC(kernel='rbf',
                      C=3,
                      gamma = 'scale',
                      decision_function_shape = 'ovr',
                      random_state = 2022)
svm_final.fit(np.concatenate((x_train, x_val), axis=0), np.concatenate((y_train, y_val), axis=0))
y_val_hat = svm_final.predict(x_test)
accuracy = accuracy_score(y_val_hat, y_test)
accuracy
                   

0.9770833333333333

In [15]:
svm_final = svm.SVC(kernel='rbf',
                      C=3,
                      gamma = 'scale',
                      decision_function_shape = 'ovr',
                      random_state = 2022)
svm_final.fit(np.concatenate((x_train, x_val), axis=0), np.concatenate((y_train, y_val), axis=0))
y_val_hat = svm_final.predict(x_test)
accuracy = accuracy_score(y_val_hat, y_test)
accuracy

pandas.core.series.Series

In [84]:
x_test.shape

(2400, 5936)

In [57]:
def score_calc():
    score = 0.0
    preds = svm_final.predict(x_test)
    i = 0
    for row in y_test:
        if str(row) == str(preds[i][0]):
            score = score + (1/3)
        i= i + 1
    return score

test_score = score_calc()
test_score

781.6666666666856

In [ ]:
import os
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [75]:
Hee_csv = pd.read_csv('30_samples_labeled_csv.csv',header=None, names=["index", "string"],delimiter=';')
Hee_csv['CC'] = 0
Hee_csv['D'] = 0
Hee_csv['Y'] = 0
Hee_csv = Hee_csv.astype(str)
# Iterate string digits and append classes
for i, row in Hee_csv.iterrows():
    if len(row['string']) != 4:
        row['CC'] = '1'
        row['D'] = '10'
        row['Y'] = '10'
    else:
        row['D'] = row['string'][2]
        row['Y'] = row['string'][3]
        if row['string'][0:2] == '18':
            row['CC']='0'
        else:
            row['CC']='1'
#os.chdir(path_images)
image_array, filename = dl.load_image_function('30_samples')

# and convert to a img_df
img_df = pd.DataFrame({'filename': filename, 'img_value': list(image_array)}, 
                        columns=['filename', 'img_value'])
# Create proper index value in img_df to allow merge on Hee_csv
img_df['index'] = img_df['filename']
for i, row in img_df.iterrows():
    row['index'] = str(img_df['index'][i]).split('.')[0]
img_df
# End of creating img_df
#
# Start: merge img_df with Hee_csv
# match index type on dataframes to merge
Hee_csv['index'] = Hee_csv['index'].astype(int)
img_df['index'] = img_df['index'].astype(int)
# Merge dataframes
df_img_classes = Hee_csv.merge(img_df)
# Rearrange order of dataframe
df_img_classes = df_img_classes.reindex(columns= ['index', 'string', 'CC', 'D', 'Y', 'img_value', 'filename'])
# End og merging data frames

In [76]:
k30_labels_CC = df_img_classes['CC']
k30_labels_D = df_img_classes['D']
k30_labels_Y = df_img_classes['Y']

In [77]:
k30_img_vals =  df_img_classes['img_value']
values = np.zeros(shape=(len(k30_img_vals),56,106,1))
i = 0
j = 0
while i < len(k30_img_vals):
    values[i] = k30_img_vals[j]
    i += 1
    j+=1
k30_img_vals = values/255

In [85]:
n_samples = len(k30_img_vals)
k30_img_vals = k30_img_vals.reshape((n_samples, -1))
k30_img_vals.shape

(1000, 5936)

In [89]:
def score_calc():
    score = 0.0
    preds = svm_final.predict(k30_img_vals)
    i = 0
    for row in k30_labels_CC:
        if str(row) == str(preds[i]):
            score = score + (1/3)
        i= i + 1
    return score

test_score = score_calc()
test_score

274.0000000000019

In [87]:
preds = svm_final.predict(k30_img_vals)
preds

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0